## Install dependencies

In [1]:
import os
import tensorflow as tf
%matplotlib inline

from data.abc import ABCPreProcessor

## Overview of Preprocessing steps

In this notebook, we will be preprocessing two types of data --> **ABC Notation** data and **Audio** data

### ABC Notation Data

- Strip away **Tune body**, **key**, **meter**, **rhythm** and store all other fields of an ABC track as metadata
- Use key and meter as conditioning symbols when generating a tune
- Tokenize according to vocabulary of musical transcription tokens

- Create a TFRecord Dataset consisting sequence examples like --> **[ tune, meter, key, rhythm ]**

### Audio Data
- Turning the full audio into short examples (4 seconds by default, but adjustable with flags)
- Inferring the fundamental frequency (or "pitch") with CREPE
- Computing the loudness features

- Create TFRecord Dataset consisting sequence examples like --> **[ Audio, f0_feature, loudness_feature ]**

#### Each tune be indexed such that using its ID, we can find its ABC Notation as well as related audio files
- A tune can be associated with more than one audio file (Different audio lengths!!)

At the end of the file, we should merge both the datasets, to obtain a single TFRecord file containing preprocessed ABC data and preprocessed audio files indexed according to the different tunes

## Initialize common variables

In [2]:
# Mention the path to the datastore
BASE_DIR = "/home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020/"
ABC_DATA_DIR = os.path.join(BASE_DIR, "datasets", "abc_data")
AUDIO_DATA_DIR = os.path.join(BASE_DIR, "datasets", "audio")
ABC_TFRECORD_DIR = os.path.join(BASE_DIR, "tfrecords", 'abc')
AUDIO_TFRECORD_DIR = os.path.join(BASE_DIR, "tfrecords", 'audio')
PROCESSED_ABC_FILENAME = 'processed-abc-files'
PROCESSED_AUDIO_FILENAME = 'processed-audio-files'

## Preprocessing - ABC Notation Dataset

#### To understand the underlying distribution of tunes in Irish music, it can be helpful to visualize the following quantities:
- Maximum length of tunes in each category
- Number of tunes in each category
- Number of tunes in each key
- Number of tunes in each meter

In [3]:
preprocessor = ABCPreProcessor(ABC_TFRECORD_DIR, PROCESSED_ABC_FILENAME)
json_path = preprocessor.process(ABC_DATA_DIR)
#preprocessor.visualize_stats()
tfrecord_path = preprocessor.save_as_tfrecord_dataset()

The raw data has already been processed. Pre-processed information found at - /home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020/tfrecords/abc/processed-abc-files.json
The TFRecord file already exists at /home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020/tfrecords/abc/processed-abc-files.tfrecord ...


In [4]:
dataset = preprocessor.load_tfrecord_dataset()
print(dataset)
for context, sequence in dataset:
    print(context)
    print(tf.sparse.to_dense(sequence['tune']))
    break

<MapDataset shapes: ({K: (), M: (), R: ()}, {tune: (None, None)}), types: ({K: tf.int64, M: tf.int64, R: tf.int64}, {tune: tf.int64})>
<MapDataset shapes: ({K: (), M: (), R: ()}, {tune: (None, None)}), types: ({K: tf.int64, M: tf.int64, R: tf.int64}, {tune: tf.int64})>
{'K': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'M': <tf.Tensor: shape=(), dtype=int64, numpy=5>, 'R': <tf.Tensor: shape=(), dtype=int64, numpy=20>}
tf.Tensor(
[[ 2  4  2 14 12 14  1  3  8  2  2  7 12  1  2  4  2  2  4  8  1  3  2  4
   8  3  5  1  2  4  2 16 14 15  1  3  8  2  2  7 12  1  7 11  7  2  3  5
   1 21  9  3  8  3  6  8 17  1  6  9  3  8  3  6  5  1  1  1 17  9  6  3
   5  3  8  1  2  3  3  8  3  5  1  9  5  3  5  3  8  1  2  4  8  3  5  9
   1 16 10 15 13 10  5  1  3  8  2  2  7 12  1  7 11  7  2  3  5  1 21  9
   3  8  3  6  5 17  1  6  9  3  8  3  6  8  1  1  1 17 16  2 15 14  6  3
   1  3 28  8  2  2  7 12  1 16  2 15  2  4  8  1  3  2  4  8  3  5  1 16
   2 15 14  6  3  1  3 28  8  2  2  7 12  1 16  7

tf.Tensor(
[[ 5  3  4 16  7 15  1 14 12 14  7  6  4  1 14 12 14 16  8 15  1  4  7 12
  12  4  3  1  5  3  4 16  7 15  1 14 12 14  7  6  4  1  2  7 11  7  4  2
   1 21  7 11 12 14  4  3 17  1  6  7 11 12 14  3  8  1  1  1 17 16  4 15
   3  4  7  1 16  4 15  3  4  7  1  2  8  4  2  7  2  1  4  7 12 12  3  8
   1 16  4 15  3  5 10  1 13 10  5  3  6  4  1  8  4  2  7  4  2  1 21  7
  11 12 14  3  8 17  1  6  7 11 12 14  4  3  1  1  1 17  5  9 10  5  9 10
   1  3 10  3  4  8  3  1  4 16  8  6 13 10 13  1 20 10  5  9  5  3  1 33
  21  5  9 10  5  9 10  1  3 10  3  4  8  3  1 13 10  9 10 20 13  1 10  9
   5  3  6 10 17  1 33  6  3  5  9 16 10 15  1 10  9  5  3  6  4  1  8  4
   2  7  4  2  1  7 11 12 14  6  3  1  1  1 17  5  3  4  7  2  4  1 16 14
  15  7  2  4  1 16 14 15  8  6  2  1  4  7 12  7  4  3  1  5  3  4  7  2
   4  1 16 14 15  7  2  4  1  2  7 11  7  4  2  1  7 11 12 14  6  3 17  1
   1 17 16  4 15  3  4  7  1  4  2  4  3  4  7  1  2  4  2 14  4 14  1  2
   7 11 12  6 14  1  4  2  

tf.Tensor(
[[16  2 15  4  2  7 11 14  1 19 15 12 11  7  2  4  8 15  2  1  7  6 12 27
   7 22 27 12  7  1  8  2  7  8  2  7 11  7  1 16  2 15  4  2  7 11 14  1
  19 15 12 11  7  2  4  8  6  8  2  1  3  9 16  9  6  5 25  8  3  4  1  2
  11  7 12 11 14 14  6 17  1  1 17  3  9 16  9  6 13  9 16  9  6  1 20  9
  13  9 10  9  5  3  1 25  8  3  5  9 10  5  8  5  1 10  5 25  8  2  7  2
   4  8  1  3  9 16  9  6 13  9 16  9  6  1  3  6  9  3  5  3 25  8  2  1
  19 15  4 25  8  3  5  8  3  5  8  4  1  2 11  7 12 14 26 17  1  1 17  2
   6  7  4  2  7 11 14  1 19 15 12 11  7  2  4  8  4  8  2  1  7  6 12  7
   8  7 12  7  1  8  7 16  7  6  2  7 11  7  1  2  6  7  4  2  7 11 14  1
  19 15 12 11  7  2  4  8  6  2  8  1  3  9 16  9  6  5 25  8  3  4  1 21
   2 11 12  7 11 14 19 15 12 11  7 17  1  6  2 11 12  7 14 26  1  1  1 17
   3  9 16  9  6 13  9 16  9  6  1 20  6 13  9 10  9  5  3  1 19 15  4 25
   8  3  5  9 10  5 16  5  6  1 10  5 25  8  2  7  2  4  8  1  3  9 16  9
   6 13  9 16  9  6  1  3  

{'K': <tf.Tensor: shape=(), dtype=int64, numpy=16>, 'M': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'R': <tf.Tensor: shape=(), dtype=int64, numpy=13>}
tf.Tensor(
[[12  4 16  4  6  3  4 16  4  6  1 12  4  4  2 11 12 11  2  1  4 12 16 12
   6  4  6  2  4  1 21  3  9  9  5  3  4  2 11 17  1  6  3  9  9  5  3  4
   2  3  1  1  4  5  5  3  5  6  3  5  1  9  5  3  5  9  5 16  5  6  1  9
   5 20  5  9  5 20  5  1  9  5  3  5  9  5  5  3  1  4  5  5  3  5  6  3
   5  1  9  5  3  5  9 13 16 13  6  1 16 20 15  9 16 13 15  9  1  5  9  9
   5  3  4  2 11  1  1]], shape=(1, 127), dtype=int64)
{'K': <tf.Tensor: shape=(), dtype=int64, numpy=9>, 'M': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'R': <tf.Tensor: shape=(), dtype=int64, numpy=13>}
tf.Tensor(
[[16  7 15 11  7  2  4  8  1  3  4  5  4  3  4  2 11  1 14 11  2 11  4 11
   2 11  1 14 11  2 11  7 11 12 11  1 16  7 15 11  7  2  4  8  1  3  4  5
   4  3  4  2  6  1  4  5 16  5  6  3  4  2 11  1 14 12 11 14 16 12 15 11
  17  1  1 17  7  4 16  4 

{'K': <tf.Tensor: shape=(), dtype=int64, numpy=22>, 'M': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'R': <tf.Tensor: shape=(), dtype=int64, numpy=13>}
tf.Tensor(
[[ 8  2  7  4  2 14 16 14  6  1  2 14 16 14  6 11  7  2  4  1  8  2  7  4
   2  7 11  7  1 21  2  3  3  5  9  3  5  3 17  1  6  2  3  3 25  8  3 15
   5  1  1  9  6  3  9  5  9 10  5  1  9  3  5  3 25  8  2 16  2  6  1  9
   6  3  9  5  3 25  8  4  1  2  3  3 25  8  3  6 25  8  3  1  5 13 16 13
   6  5  9 10  5  1  3  9  5  3 28  8  2  2  7  1  2  3  3 25  8  3 15 28
   8  1  2  3  3  5  9  3  5  3  1  1  8  2  7  4 16  2 15  3  1 16  4 15
   7 16  2 15  3  1  8  2  7  4  2  7 11  7  1  2  3  3  5  9  3  5  3  1
   8  2  7  4 16  2 15  3  1  4  7  7 11  7  4  3  4  1  8  2  7  4  2  7
  11  7  1  2  3  3 25  8  3 15  5  1  1 19 15  9  5  3 13  3  9  3 13  3
   1 19 15  9  5  3 13  9 10  5 16  5  6  1  9  3 13  3 20  3 13  3  1  9
  13 10  5  3  8  2  8  1  3  5  9 10 16 13 15 20  1 13 10  9 13 16 10 15
   5  1  9  5  3  9  5

...

{'K': <tf.Tensor: shape=(), dtype=int64, numpy=7>, 'M': <tf.Tensor: shape=(), dtype=int64, numpy=5>, 'R': <tf.Tensor: shape=(), dtype=int64, numpy=12>}
tf.Tensor(
[[ 4  1  2 11 14 14 11  2  1  8  3  8  2  7 11  1 19 12 18 11 18  7 23 19
  14 27 23 14 27  1 12 11  7  2  8 28  4  1  2 11 14 14 11 19  7  1  2 23
  28  4 25  8  3  6  5  1  9  5  3  5  3 19  8  1  2 23  3  3  3  6 17  1
   1  1  5  1  9  5  3 13  9  3  1 19  3 18  5 18  9 23 19 10 13 23  9  3
   1  5  3  8 10  5  8  1 19  8 18  3 18  5 23 19  9 10 23  9  5  1  9  5
   3  5  3  8  1 19  3 18 10 18  9 23 19  3  8 23  2  7  1 11  7  2  8 28
   4 19  8  1  2 23  3  3  3  6 17  1]], shape=(1, 156), dtype=int64)
{'K': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'M': <tf.Tensor: shape=(), dtype=int64, numpy=5>, 'R': <tf.Tensor: shape=(), dtype=int64, numpy=12>}
tf.Tensor(
[[10  1  9  6  3  5  3  8  1  3  6  4  2  7 11  1  7  2  4  3  2 19 11  1
   7 23 12 12 12  6 10  1  9  6  3  5  3  8  1  3  6  4  2  7 11  1  7  2
   4  3  2 1

tf.Tensor(
[[ 7  4  3  8  2 11  1  7 10  5  3  4  7  1  8  5  8  4  3  4  1  2  4  7
  11 12 14  1  7  4  3  8  2 11  1  7 10  5  3  4  7  1  8  5  8  4 13 10
   1 21  9  5  9 10 15 17  1  6  9  5  9 10  6  3  1  1  1 17 10  3  4 10
   3  4  1 10  9 13 10  3  4  1 10  9 10  5 13 10  1  9  3 25  8  3  5  9
   1 10 20 10  9 13  9  1  5 10  5  3  5  9  1 10  3  4  2  7  2  1 21  4
   7 11  7  6  3 17  1  6  4  7 11  7 15  1  1  1 17  7  6  4  8  2 11  1
   7 10  5  3  4  7  1  8 15 16  4 15  1  2  4  7 11 12 14  1  7  6  4  8
   2 11  1  7 10  5  3  4  7  1  8  4  2  4  3 10  1 21  9  5  9 10 15 17
   1  6  9  5  9 10  6  9  1  1  1 17 10  3  4 10  3  4  1 10 13  9 10  3
   4  1 10  9 10  5 13 10  1  9  3  3  3  5  9  1 16 10 15 16  9 15  1  5
   9  5  3  5  9  1 10  3  4  2  4  8  1 21  4  7  7  7  6  9 17  1  6  4
   7  7  7 15  1  1]], shape=(1, 270), dtype=int64)
{'K': <tf.Tensor: shape=(), dtype=int64, numpy=9>, 'M': <tf.Tensor: shape=(), dtype=int64, numpy=5>, 'R': <tf.Tensor: shape

{'K': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'M': <tf.Tensor: shape=(), dtype=int64, numpy=3>, 'R': <tf.Tensor: shape=(), dtype=int64, numpy=13>}
tf.Tensor(
[[16 13 15  9 16 10 15  5  1  9  5  3  9  5  8  2 20  1 13  3 16  3  6 20
   3 13  3  1  8  3  5  9 10  6  9 10  1 13 10  9 13 10  9  5 10  1  9  5
   3  9  5  8  2  4  1  8  2  5  2  9  2  5  9  1 21 10  5  8  5  3  5  9
  10 17  1  6 10  5  8  5  3  4  2  7  1  1  1 17 11  2  3  5 28  9  6  3
   9  1  5 28  8 16  8  6  7 12  7  8  1  2 14 11  2  3  5  3  4  1  8  5
  13 10  9  3  2  7  1 11  2  3  5 28  9  6  3  9  1  5 28  8 16  8  6 10
   8  5 10  1 13 10  5  3  8  2  5  9  1 21 10  5  8  5  3  4  2  7 17  1
   6 10  5  8  5  3  5  9 10  1  1  1 17 13  3 16  3  6 10  5 16  5  6  1
   9  3 16  3  6  5  8  2  6  1 13  3 16  3  6 20  3 13  3  1  8  3  5  9
  10  5  9 10  1 13 10  9 13 10  9  5 10  1  9  5  3  9  5  8  2  4  1  8
   2  5  2  9  6  5  9  1 21 10  5  8  5  3  5  9 10 17  1  6 10  5  8  5
   3  6  2  7  1  1  1 

tf.Tensor(
[[ 8  3 18  8 18  4  8 18  4 18  1  2  4 18  2 18  7 24  2  1  4  3  5  3
   1 10 13 18 10 18  5  3  1  5 13 10 24  5  1  3  4  7  2 18  4 18  1  8
   5  3  4  1  2  6  2  6 17  1  1 17  5 13 13 10 18  5 18  1  3 10 10  3
   1  5 13 13 20  1 10 13 18 10 18  5  3  1  5 13 10 24  5  1  3  4  7  2
  18  4 18  1  8  5  3  4  1  2  6  2  6 17  1  1 17  8  6  4  6  1  2  4
  18  2 18  7 24  2  1  4  3  5  3  1 10  6 10  9  1  5 13 10 18 13 18 10
  18  5 18  1  3  4  7  2 18  4 18  1  8  5  3  4  1 21  2  6  2  4 17  1
   6  2  6  2  4 18  3 18  1  1  1 17  5 13 13 10 18  5 18  1  3 10 10  3
   1  5 13 13 20  1 10  6 10  9  1  5 13 10 18 13 18 10 18  5 18  1  3  4
   7  2 18  4 18  1  8  5  3  4  1 21  2  6  2  4 18  3 18 17  1  6  2  6
   2  4  1  1]], shape=(1, 244), dtype=int64)
{'K': <tf.Tensor: shape=(), dtype=int64, numpy=2>, 'M': <tf.Tensor: shape=(), dtype=int64, numpy=8>, 'R': <tf.Tensor: shape=(), dtype=int64, numpy=19>}
tf.Tensor(
[[11 24 14 11  2  1  7 24 11  7  4  1  2

tf.Tensor(
[[ 4  6  7  6  1  3  5 18  9 18 10  4  1  2  6  2  4  1  3  5 18  3 18  4
   2  1  4  6  7  6  1  3  5 18  9 18 10  4  1  2  2  4  2  1 21  7 11  7
   2 17  1  6  7 11  7 10  1  1  1 17  9 13  3  6  1  5 24  3  4  2  1  4
  24  2  7 12  1  4 24  2  7 12  1  9 13  3  6  1  5 24  3  4  2  1  4 24
   2  7  4 18  3 18  1 10  6 10  6 17  1]], shape=(1, 109), dtype=int64)
{'K': <tf.Tensor: shape=(), dtype=int64, numpy=9>, 'M': <tf.Tensor: shape=(), dtype=int64, numpy=8>, 'R': <tf.Tensor: shape=(), dtype=int64, numpy=19>}
tf.Tensor(
[[ 4  3  8  2  1 14  6  4  6  1 14  6  2  6  1 14  6  4 24  2  1  4  3  8
   2  1 14  6  4 24  2  1  4  3  8  2  1 21  7  6  7  2 17  1  6  7  6  7
   4  1  1  1 17  5  6  5  9 18  5 18  1  3  6  3  5  1  9  6  9  3  1 10
   6 10  9  1  5  6  5 24  9  1  5  3  3  8  1  4 18  8 18  3  5  9  1 10
   3  5  9  1 10  6  3 24  8  1  4  7  7  2 18  7 18  1 11  2  2  4 18  2
  18  1  7  2  4  3  1 10  6  3  6  1  4  6  4  2  1  4  3  8  2  1 21  7
   6  7  4 17